In [1]:
import os
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())

21/10/23 19:12:14 WARN Utils: Your hostname, Sumans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
21/10/23 19:12:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/23 19:12:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.3
      /_/

Using Python version 3.8.2 (default, Apr  8 2021 23:19:18)
SparkSession available as 'spark'.


#### First let's read in the processed and cleansed data

In [2]:
sc.addFile('/Users/sumangangopadhyay/complex-spark-transformations/config.py')

In [3]:
import config as cf
from pyspark.sql import functions as func
from pyspark.sql.types import IntegerType

In [4]:
clean_data_path = cf.clean_data_path()
primary_response_variables = cf.primary_response_variables().split(',')
secondary_response_variables = cf.secondary_response_variables().split(',')
primary_explanatory_variables = cf.primary_explanatory_variables().split(',')

In [5]:
df = spark.read.csv(clean_data_path, header=True)

#### Let us look for seasonality, i.e. if the incidents of violation are more in particular month(s). Note how the Issue_Month column is sorted using type casting, since this column wasn't derived as an integer before

In [6]:
df.groupBy(df.Issue_Month).count().orderBy(df.Issue_Month.cast(IntegerType())).show()

21/10/23 19:12:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-------+
|Issue_Month|  count|
+-----------+-------+
|          1| 877365|
|          2| 826967|
|          3| 964737|
|          4| 888402|
|          5|1020244|
|          6| 852187|
|          7|    370|
|          8|    309|
|          9|    367|
|         10|    274|
|         11|    338|
|         12|    358|
+-----------+-------+



#### It is quite clear that the violations are most common during the first half of the year, so let's dig deeper in the first half

In [7]:
df_first6months = df.filter((df.Issue_Month.cast(IntegerType()) >= 1) \
                           | (df.Issue_Month.cast(IntegerType()) <= 6))

In [8]:
df_temp = df_first6months.groupBy(df_first6months.Issue_DayofMonth).count()

In [9]:
df_temp.columns

['Issue_DayofMonth', 'count']

In [10]:
df_temp.createOrReplaceTempView('t_count_per_day_of_month')

In [11]:
df_temp_standardised = spark.sql("select \
                                 t1.Issue_DayofMonth, t1.count \
,round((t1.count - t2.avg_issue_count)/t2.stddev_issue_count,0) as standardised_issue_count \
,round((t1.count - t2.min_count)/(t2.max_count - t2.min_count),0) as normalised_issue_count \
from t_count_per_day_of_month t1 \
join \
(select avg(count) as avg_issue_count, stddev(count) as stddev_issue_count, min(count) as min_count \
,max(count) as max_count from t_count_per_day_of_month)t2 \
on 1=1")

In [12]:
df_temp_standardised.orderBy(df_temp_standardised.Issue_DayofMonth.cast(IntegerType())).show(40)

+----------------+------+------------------------+----------------------+
|Issue_DayofMonth| count|standardised_issue_count|normalised_issue_count|
+----------------+------+------------------------+----------------------+
|               1|192311|                     1.0|                   1.0|
|               2|195915|                     1.0|                   1.0|
|               3|225398|                     2.0|                   1.0|
|               4|179977|                     0.0|                   1.0|
|               5|162463|                     0.0|                   1.0|
|               6|211419|                     1.0|                   1.0|
|               7|179266|                     0.0|                   1.0|
|               8|187606|                     0.0|                   1.0|
|               9|170336|                     0.0|                   1.0|
|              10|182476|                     0.0|                   1.0|
|              11|166438|             

#### From the Standardised scores of Days of Months above, it seems that the violations are more common in the start and end of the months

In [13]:
df.columns

['Summons_Number',
 'Plate_ID',
 'Registration_State',
 'Plate_Type',
 'Violation_Code',
 'Vehicle_Body_Type',
 'Vehicle_Make',
 'Issuing_Agency',
 'Street_Code1',
 'Street_Code2',
 'Street_Code3',
 'Vehicle_Expiration_Date',
 'Violation_Location',
 'Violation_Precinct',
 'Issuer_Precinct',
 'Issuer_Code',
 'Issuer_Command',
 'Issuer_Squad',
 'Violation_Time',
 'Time_First_Observed',
 'Violation_County',
 'Violation_In_Front_Of_Or_Opposite',
 'House_Number',
 'Street_Name',
 'Intersecting_Street',
 'Date_First_Observed',
 'Law_Section',
 'Sub_Division',
 'Violation_Legal_Code',
 'Vehicle_Color',
 'Vehicle_Year',
 'Meter_Number',
 'Feet_From_Curb',
 'Violation_Post_Code',
 'Violation_Description',
 'Violation_AM_or_PM',
 'Issue_Date',
 'Issue_Year',
 'Issue_Month',
 'Issue_DayofWeek',
 'Issue_DayofMonth',
 'Violation_Time_Hour']

In [14]:
df_temp = df_first6months.groupBy(df_first6months.Issue_DayofWeek).count()

In [15]:
df_temp.createOrReplaceTempView('t_count_per_day_of_week')

In [16]:
df_temp.columns

['Issue_DayofWeek', 'count']

In [17]:
df_temp_standardised = spark.sql("select \
                                 t1.Issue_DayofWeek, t1.count \
,round((t1.count - t2.avg_issue_count)/t2.stddev_issue_count,0) as standardised_issue_count \
,round((t1.count - t2.min_count)/(t2.max_count - t2.min_count),0) as normalised_issue_count \
from t_count_per_day_of_week t1 \
join \
(select avg(count) as avg_issue_count, stddev(count) as stddev_issue_count, min(count) as min_count \
,max(count) as max_count from t_count_per_day_of_week)t2 \
on 1=1")

In [18]:
df_temp_standardised.orderBy(df_temp_standardised.Issue_DayofWeek.cast(IntegerType())).show(40)

+---------------+------+------------------------+----------------------+
|Issue_DayofWeek| count|standardised_issue_count|normalised_issue_count|
+---------------+------+------------------------+----------------------+
|              1|218238|                    -2.0|                   0.0|
|              2|869368|                     0.0|                   1.0|
|              3|961584|                     1.0|                   1.0|
|              4|927707|                     1.0|                   1.0|
|              5|962045|                     1.0|                   1.0|
|              6|932620|                     1.0|                   1.0|
|              7|560356|                    -1.0|                   0.0|
+---------------+------+------------------------+----------------------+



#### Quite expectedly, the violations are more during the weekdays

#### Let us now switch our attention towards a more fine-grained view to see if the violations of specific types (Laws) are being committed by specific type of vehicles (based on their registration states) at specific places (counties where these violations occured)

In [42]:
df_temp = df_first6months.groupBy(df_first6months.Issue_DayofWeek,\
                                 df_first6months.Violation_County,\
                                 df_first6months.Law_Section,\
                                 df_first6months.Registration_State)\
.count()

In [54]:
df_pivot = df_temp.groupBy('Law_Section','Violation_County','Registration_State').pivot('Issue_DayofWeek')\
.agg(func.sum('count').alias('count'))
df_pivot = df_pivot.dropna(how='any')
df_pivot.show(100)

+-----------+----------------+------------------+-----+-----+-----+-----+-----+-----+-----+
|Law_Section|Violation_County|Registration_State|    1|    2|    3|    4|    5|    6|    7|
+-----------+----------------+------------------+-----+-----+-----+-----+-----+-----+-----+
|        408|              BX|                CT|  621| 1767| 1804| 1533| 1839| 1889| 1801|
|        408|               Q|                ND|    1|    2|   11|    5|    7|    8|    5|
|       1111|              BX|                NC|   26|   19|   24|   26|   25|   34|   25|
|       1111|              BX|                VA|   26|   33|   37|   35|   37|   38|   37|
|        408|               R|                NJ|  141|  529|  516|  616|  549|  546|  307|
|        408|              BX|                IA|    3|   23|   30|   20|   22|   26|   18|
|       1111|              BX|                CA|    4|    5|    8|    2|    8|    5|    5|
|        408|               K|                WA|   34|  232|  265|  169|  220| 

In [25]:
df_temp.groupBy('Law_Section').pivot('Issue_DayofWeek').sum('count').show(100)

+-----------+------+------+------+------+------+------+------+
|Law_Section|     1|     2|     3|     4|     5|     6|     7|
+-----------+------+------+------+------+------+------+------+
|       1111| 34443| 35862| 34880| 38101| 39287| 42399| 33278|
|        408|183795|716190|798817|742957|791164|750898|527078|
|         40|  null|  null|  null|  null|  null|     1|  null|
|       1180|  null|117314|127887|146649|131594|139321|  null|
|       6408|  null|     2|  null|  null|  null|  null|  null|
|       3040|  null|  null|  null|  null|  null|     1|  null|
+-----------+------+------+------+------+------+------+------+



In [26]:
df_temp.groupBy('Violation_County').pivot('Issue_DayofWeek').sum('count').show(100)

+----------------+-----+------+------+------+------+------+------+
|Violation_County|    1|     2|     3|     4|     5|     6|     7|
+----------------+-----+------+------+------+------+------+------+
|               K|48271|180749|206674|171812|206556|184264|126507|
|              MN| 2361|  6931|  6944|  7133|  7213|  7925|  2414|
|               Q|31438|149151|158740|159233|163532|155902|121437|
|              BX|32732|109182|118499|110442|119448|121496| 77730|
|            null| 1660|  2696|  2936|  2767|  2847|  2446|  1737|
|              QN|13373| 49600| 53168| 63327| 56714| 62787| 13225|
|             QNS| null|  null|  null|  null|  null|     1|  null|
|              BK|12231| 66244| 70032| 78169| 73213| 73937| 10759|
|           BRONX| null|  null|  null|  null|     1|  null|  null|
|               R| 3821| 10320| 10054| 10834| 10193|  9816|  7960|
|              NY|69942|283391|323105|311528|309883|300884|195832|
|              ST| 2409| 11104| 11432| 12462| 12445| 13162|  2

In [55]:
df_temp.groupBy('Registration_State').pivot('Issue_DayofWeek').sum('count').show(100)

+------------------+------+------+------+------+------+------+------+
|Registration_State|     1|     2|     3|     4|     5|     6|     7|
+------------------+------+------+------+------+------+------+------+
|                AZ|   660|  1714|  2010|  2086|  2116|  2088|  1705|
|                SC|   498|  1765|  1784|  1541|  1843|  1767|  1197|
|                NS|    18|    64|    46|    39|    69|    48|    38|
|                LA|    80|   296|   298|   218|   328|   302|   167|
|                MN|    97|  1929|  1676|  1636|  1671|  1897|  1177|
|                NJ| 20403| 74933| 83198| 81906| 82570| 81754| 51061|
|                DC|   166|   308|   287|   180|   299|   365|   324|
|                OR|    76|   504|   518|   434|   535|   431|   124|
|                99|  1290|  2510|  2964|  2575|  2673|  2361|  1682|
|                VA|  2066|  5695|  5625|  4752|  5669|  6026|  4534|
|                RI|   417|   942|   961|   736|   910|   950|   898|
|                WY|